This notebook demonstrates an end to end example of compiling a circuit to its pauli-check variant, mapping to an IBM QPU, and executing it on a noisy simulator. 

In [1]:
import numpy as np

from quantem.utils import (
    convert_to_PCS_circ_largest_clifford,
    get_VF2_layouts,
)
from quantem.pauli_checks import postselect_counts

## Generate circuit

In [2]:
from qiskit.circuit import QuantumCircuit


def hydrogen_trial_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits)
    # prepare the Hartree-Fock state
    qc.x(0)
    qc.x(1)

    qc.rx(np.pi / 2, 0)
    qc.h(1)
    qc.h(2)
    qc.h(3)

    # qc.barrier()
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.cx(2, 3)
    # qc.barrier()

    qc.rz(1.0, 3)

    qc.cx(2, 3)
    qc.cx(1, 2)
    qc.cx(0, 1)

    qc.rx(-np.pi / 2, 0)
    qc.h(1)
    qc.h(2)
    qc.h(3)

    return qc

In [ ]:
circ = hydrogen_trial_circuit(4)
circ.draw("mpl", scale=0.6)

## Apply checks to larget Clifford block

In [ ]:
from qiskit_addon_utils.slicing import combine_slices, slice_by_depth

slices = slice_by_depth(circ, 1)
combined_slices = combine_slices(slices, include_barriers=True)
combined_slices.draw("mpl", scale=0.6)

In [ ]:
num_qubits = 4
num_checks = 2

sign_list, pcs_circ = convert_to_PCS_circ_largest_clifford(circ, num_qubits, num_checks)

In [ ]:
print(sign_list)
pcs_circ.draw("mpl", scale=0.6, fold=-1)

## Select backend

In [7]:
from qiskit_ibm_runtime.fake_provider import *

fake_backend = FakeWashingtonV2()
fake_coupling_map = fake_backend.configuration().coupling_map

In [ ]:
from qiskit.visualization import plot_error_map

plot_error_map(fake_backend)

## Map to backend using VF2 algorithm 

In [ ]:
VF2_mapping_ranges, small_qc = get_VF2_layouts(circ, fake_backend)
print(len(VF2_mapping_ranges))
print(VF2_mapping_ranges)

mapping_range = VF2_mapping_ranges[0]
print(mapping_range)

In [ ]:
pcs_VF2_mapping_ranges, pcs_small_qc = get_VF2_layouts(pcs_circ, fake_backend)
print(len(pcs_VF2_mapping_ranges))
print(pcs_VF2_mapping_ranges)

pcs_mapping_range = pcs_VF2_mapping_ranges[0]
print(pcs_mapping_range)

## Execute on noisy simulator

In [11]:
import qiskit_aer.noise as noise
from qiskit_aer.primitives import SamplerV2 as sampler

basis_gates = fake_backend.configuration().basis_gates
noise_model = noise.NoiseModel.from_backend(fake_backend)

noisy_sampler = sampler(options={"backend_options": {"noise_model": noise_model}})
ideal_sampler = sampler()

In [12]:
circ.measure_all()
pcs_circ.measure_all()

In [ ]:
pcs_circ.draw("mpl", fold=-1)

In [ ]:
circ.draw("mpl", fold=-1)

In [ ]:
from qiskit import transpile

pcs_circ_transpiled = transpile(
    pcs_circ,
    basis_gates=basis_gates,
    initial_layout=pcs_mapping_range,
    optimization_level=1,
)
circ_transpiled = transpile(
    circ, basis_gates=basis_gates, initial_layout=mapping_range, optimization_level=1
)

noisy_job_result = noisy_sampler.run(
    [pcs_circ_transpiled, circ_transpiled], shots=10000
).result()

pcs_circ_counts = noisy_job_result[0].data.meas.get_counts()
noisy_circ_counts = noisy_job_result[1].data.meas.get_counts()

ideal_job_result = ideal_sampler.run([circ], shots=10000).result()
ideal_circ_counts = ideal_job_result[0].data.meas.get_counts()
postselected_counts = postselect_counts(pcs_circ_counts, sign_list, num_ancillas=2)

print(pcs_circ_counts)
print(postselected_counts)

In [ ]:
from qiskit.quantum_info import hellinger_fidelity

raw_fidelity = hellinger_fidelity(ideal_circ_counts, noisy_circ_counts)
mitigated_fidelity = hellinger_fidelity(ideal_circ_counts, postselected_counts)
postselection_rate = sum(postselected_counts.values()) / sum(pcs_circ_counts.values())

print(f"raw fidelity: {raw_fidelity}")
print(f"mitigated fidelity: {mitigated_fidelity}")
print(f"postselection rate: {postselection_rate}")